# Imports

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# %matplotlib qt
import snmfem.generate_data as gd
from snmfem.models import EDXS
import numpy as np
import hyperspy.api as hs
import matplotlib.pyplot as plt
from snmfem.conf import DB_PATH

# Parameters

In [ ]:
# To save the dataset
filename = "aspim037_N100_2ptcls_brstlg"
abs_db_path = None
# abs_db_path = "Data/wernisch_abs.json"
abs_elt_dict = None

# Continuum X-rays parameters
# They were determine by fitting experimental data from 0.6 to 18 keV. Since low energies were incorporated, the model is only effective and not quantitative. 
brstlg_pars = {"c0" : 0.094, "c1" : 1417, "c2" : 1e-6, "b0" : 1.2, "b1" : -0.06, "b2" : 0.00683}

# brstlg_pars = {"c0" : 9.3144e-04,"c1" : 1.23453632, "c2" : 4.9488e-10, "b0" : 0.14986191, "b1" : -0.00188483, "b2" : 2.5428e-04}


scale = 1

# Average number of counts in one spectrum of the artificial data
N = 100 

# Creation of the pure spectra of the different phases.
phase1 = EDXS(DB_PATH,abs_db_path,brstlg_pars)
#phase1.generate_abs_coeff({"8":1.0,"12" : 0.51,"14":0.61,"13":0.07,"20":0.04,"62":0.02,"26":0.028,"60":0.002,"71":0.003,"72":0.003,"29" : 0.02})
# Gaussians corresponding to elements
phase1.generate_spectrum({"8":1.0,"12" : 0.51,"14":0.61,"13":0.07,"20":0.04,"62":0.02,"26":0.028,"60":0.002,"71":0.003,"72":0.003,"29" : 0.02},scale)

phase2 = EDXS(DB_PATH,abs_db_path,brstlg_pars)
#phase2.generate_abs_coeff({"8":0.54,"26":0.15,"12" : 1.0,"29":0.038,"92":0.0052,"60":0.004,"31":0.03,"71":0.003})
phase2.generate_spectrum({"8":0.54,"26":0.15,"12" : 1.0,"29":0.038,"92":0.0052,"60":0.004,"31":0.03,"71":0.003},scale)

phase3 = EDXS(DB_PATH,abs_db_path,brstlg_pars)
#phase3.generate_abs_coeff({"8":1.0,"14":0.12,"13":0.18,"20":0.47,"62":0.04,"26":0.004,"60":0.008,"72":0.004,"29":0.01})
phase3.generate_spectrum({"8":1.0,"14":0.12,"13":0.18,"20":0.47,"62":0.04,"26":0.004,"60":0.008,"72":0.004,"29":0.01},scale)

# Objects needed for the creation of data
# list of spectra
phases = np.array([phase1.spectrum,phase2.spectrum,phase3.spectrum])
# list of densities which will give different total number of events per spectra
densities = np.array([1.0,1.33,1.25])

In [ ]:
phases.shape

# Artificial data creation

Note : Each function of ArtificialSpim updates the current object and adds on top. It is usually better to re-create a new object to change the artificial data settings.
Spim corresponds to spectrum image.

In [ ]:
# Creation of an ArtificialSpim instance
spim=gd.AritificialSpim(phases,densities,(80,80))
assert(spim.phases.shape==(3, 1980))
assert(spim.densities.shape==(3,))

In [ ]:
np.testing.assert_allclose(np.sum(spim.phases, axis=1), np.ones([3]))

In [ ]:
# We add two particles belonging to two different phases to the data
spim.sphere((25,30),3.5,3.5,0.0,0.5,1)
spim.sphere((55,30),3.5,3.5,0.0,0.5,2)

In [ ]:
# Generates a noiseless version of the dataset
spim.generate_spim_deterministic()
assert(spim.generated_spim.shape==(80, 80, 1980))

In [ ]:
np.testing.assert_allclose(np.sum(spim.generated_spim, axis=2), np.ones([80,80]))

In [ ]:
D = spim.phases.T
A = spim.weights.reshape(-1, 3).T
X = spim.generated_spim.reshape(-1, 1980).T
np.testing.assert_allclose(D@A, X)

In [ ]:
plt.figure(figsize=(15, 4))
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.imshow(spim.weights[:,:,i], vmin=0, vmax=1)
    plt.colorbar()

In [ ]:
# Generate noisy spectra based on a Poisson distribution with average N
spim.generate_spim_stochastic(N)

In [ ]:
# plt.imshow(spim.density_map)
# plt.colorbar()

In [ ]:
assert(spim.continuous_spim.shape==(80, 80, 1980))

In [ ]:
D = spim.phases.T
A = spim.weights.reshape(-1, 3).T
w = spim.densities
Xdot = spim.continuous_spim.reshape(-1, 1980).T
# n D W A
np.testing.assert_allclose(N*D@np.diag(w)@A, Xdot)

In [ ]:
spim.densities

In [ ]:
# ns = [10, 30, 100, 300, 1000, 3000, 10000, 30000]
# rels = []
# for n in ns:
#     spim.generate_spim_stochastic(n)
#     Xhat = spim.continuous_spim
#     Xtilde = spim.stochastic_spim
#     r = np.linalg.norm((Xhat-Xtilde).flatten())/np.sqrt(np.linalg.norm(Xhat.flatten())*np.linalg.norm(Xtilde.flatten()))
#     print("For {}, the relative error is: {}".format(n, r))
#     rels.append(r)

In [ ]:
# Generate noisy spectra based on a Poisson distribution with average N
spim.generate_spim_stochastic(N, old=True)
Xhat2 = spim.continuous_spim
spim.generate_spim_stochastic(N, old=False)
Xhat = spim.continuous_spim

In [ ]:
np.linalg.norm((Xhat-Xhat2).flatten())/np.sqrt(np.linalg.norm(Xhat.flatten())*np.linalg.norm(Xhat2.flatten()))

In [ ]:
spim.generate_spim_stochastic(N, old=False)
Xhat = spim.continuous_spim
# To save the data and the ground truth under similar names
spim.save(filename)

# Data visualisation

In [ ]:
# To visualise the pure spectrum of the first phase
x = phase1.x
plt.plot(x,phases[2],"g-",linewidth=2)
plt.plot(x,phase1.continuum_xrays()*scale)
plt.xlabel("Energy [keV]")
plt.ylabel("Intensity")
plt.rcParams.update({'font.size': 22})
plt.tight_layout()

## Interactive visualization of the created datacube

3 windows will pop : 

1) Map of the sum of counts per pixel - A very tiny green square will be plotted as well at the top left

2) Spectrum of the top left corner pixel (hyperspy built-in)

3) Sum of spectra of the green area in 1)

It possible to move and reshape the green area in 1) to explore the data

In [ ]:
signal=hs.signals.Signal1D(spim.stochastic_spim)
roi=hs.roi.RectangularROI(12,12,24,24)
signal.plot()
spim_ROI=roi.interactive(signal)
    
sum_ROI=hs.interactive(spim_ROI.sum,
               event=spim_ROI.axes_manager.events.any_axis_changed,
               recompute_out_event=None)
sum_ROI.plot()